In [21]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import locale
locale.setlocale(locale.LC_ALL, 'ID')
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from requests.exceptions import ConnectionError
import unicodedata
import mysql.connector
import pandas as pd

In [22]:
con = mysql.connector.connect(user='root', password='', host='127.0.0.1', database='news_db')

In [55]:
df = pd.read_sql("select * from article", con)

In [129]:
df.head()

,id,post_id,author,pubdate,category,subcategory,content,comments,images,title,tags,url,source,is_popular,tokenized_text,tokenized_text_wo_punc,tokenized_text_2
0,1,52050,Adi Hidayat,2018-08-09 09:33:00,Otomotif,Berita Otomotif,TANGERANG – Honda Brio diberikan diskon sebesa...,0,https://content.icarcdn.com/styles/article_cov...,Honda Brio Lama Hanya Dapat Diskon Rp 10 Juta,"Berita Otomotif, Honda-NCSR, Honda, Honda Indo...",https://www.mobil123.com/berita/honda-brio-lam...,mobil123,0,"[TANGERANG, –, Honda, Brio, diberikan, diskon,...","[TANGERANG, Honda, Brio, diberikan, diskon, se...","[(, , ), (, , ), (, , ), (, , ), (, , ), (, , ..."
1,2,52051,Adi Hidayat,2018-08-09 09:32:00,Otomotif,Berita Otomotif,TANGERANG –Suzuki SX4 S-Crosshadir dengan disk...,0,https://content.icarcdn.com/styles/article_cov...,Suzuki SX4 S-Cross Didiskon Rp 30 Juta,"Berita Otomotif, Suzuki-NCSR, Suzuki, Suzuki I...",https://www.mobil123.com/berita/suzuki-sx4-s-c...,mobil123,0,"[TANGERANG, –Suzuki, SX4, S-Crosshadir, dengan...","[TANGERANG, dengan, diskon, hingga, Rp, juta, ...","[(, , ), (, , ), (, , ), (, -Crosshadir, ), (,..."
2,3,52048,Insan Akbar Krisnamusi,2018-08-09 09:29:00,Otomotif,Berita Otomotif,TANGERANG – Honda memanfaatkan Gaikindo Indone...,0,https://content.icarcdn.com/styles/article_cov...,Honda Peragakan Fitur Keselamatan Lewat Perang...,"Berita Otomotif, Honda, Honda Indonesia, PT. H...",https://www.mobil123.com/berita/honda-peragaka...,mobil123,0,"[TANGERANG, –, Honda, memanfaatkan, Gaikindo, ...","[TANGERANG, Honda, memanfaatkan, Gaikindo, Ind...","[(, , ), (, , ), (, , ), (, , ), (, , ), (, , ..."
3,4,52047,Adi Hidayat,2018-08-09 09:25:00,Otomotif,Berita Otomotif,TANGERANG –SuzukiErtiga dijual dengan diskon h...,0,https://content.icarcdn.com/styles/article_cov...,Suzuki Ertiga Didiskon Rp 24 juta,"Berita Otomotif, Suzuki-NCSR, Suzuki Indonesia...",https://www.mobil123.com/berita/suzuki-ertiga-...,mobil123,0,"[TANGERANG, –SuzukiErtiga, dijual, dengan, dis...","[TANGERANG, dijual, dengan, diskon, hingga, Rp...","[(, , ), (, , ), (, , ), (, , ), (, , ), (, , ..."
4,5,52046,Insan Akbar Krisnamusi,2018-08-09 09:20:00,Otomotif,Berita Otomotif,TANGERANG – Nissan kembali ke sport utility ve...,0,https://content.icarcdn.com/styles/article_cov...,Nissan Kembali ke SUV Berdesain Kotak Lewat Terra,"Berita Otomotif, Nissan, Nissan Indonesia, PT....",https://www.mobil123.com/berita/nissan-kembali...,mobil123,0,"[TANGERANG, –, Nissan, kembali, ke, sport, uti...","[TANGERANG, Nissan, kembali, ke, sport, utilit...","[(, , ), (, , ), (, , ), (, , ), (, , ), (, , ..."


In [57]:
import nltk
from nltk import word_tokenize
from nltk.util import ngrams
from collections import Counter

In [59]:
# nltk.download('punkt')

In [60]:
# df['tokenized_text'] = df['content'].apply(word_tokenize)

In [44]:
print(df.iloc[7]['url'])

https://www.mobil123.com/berita/first-impression-wuling-suv-kuda-hitam-di-kelas-suv-tahun-depan/52045


In [61]:
df['tokenized_text_wo_punc'] = df['tokenized_text'].apply(lambda tokens: [word for word in tokens if word.isalpha()])

In [62]:
df.iloc[9]['tokenized_text_wo_punc']

['TANGERANG',
 'HadirnyaSuzuki',
 'Swift',
 'Strong',
 'Hybriddi',
 'Gaikindo',
 'Indonesia',
 'International',
 'Auto',
 'Show',
 'GIIAS',
 'merupakan',
 'langkah',
 'lanjutan',
 'dari',
 'PT',
 'Suzuki',
 'Indomobil',
 'Sales',
 'SIS',
 'untuk',
 'menghadirkan',
 'kendaraan',
 'ramah',
 'lingkungan',
 'di',
 'Tanah',
 'gelaran',
 'GIIAS',
 'kali',
 'ini',
 'Suzuki',
 'Indonesia',
 'menampilkan',
 'teknologi',
 'terkininya',
 'strong',
 'hybrid',
 'pada',
 'diri',
 'Suzuki',
 'Swift',
 'Memang',
 'bukan',
 'yang',
 'pertama',
 'karena',
 'PT',
 'SIS',
 'sendiri',
 'sempat',
 'melansir',
 'teknologi',
 'hybrid',
 'ringan',
 'mild',
 'hybrid',
 'pada',
 'Suzuki',
 'Ertiga',
 'strong',
 'hybrid',
 'Suzuki',
 'sekarang',
 'sudah',
 'menerapkan',
 'dua',
 'perangkat',
 'tambahan',
 'yaitu',
 'motor',
 'generator',
 'unit',
 'MGU',
 'dan',
 'baterai',
 'lithium',
 'Dengan',
 'kemajuan',
 'teknologi',
 'ini',
 'Suzuki',
 'Swift',
 'Strong',
 'Hybrid',
 'siap',
 'eksis',
 'apabila',
 'nantiny

In [63]:
print(df.iloc[9]['url'])

https://www.mobil123.com/berita/suzuki-swift-strong-hybrid-siap-eksis-di-tanah-air/52042


In [92]:
pattern = r'\w+'
pattern2 = r'.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)'

In [68]:
df['tokenized_text_2'] = df['content'].apply(lambda x: nltk.regexp_tokenize(x, pattern))

In [102]:
list1 = nltk.regexp_tokenize(df.iloc[9].content, pattern)

In [103]:
list1 = ' '.join([' '.join(nltk.regexp_tokenize(word, pattern2)) for word in list1])

In [104]:
list1 = nltk.word_tokenize(list1.lower())

In [105]:
list1

['tangerang',
 'hadirnya',
 'suzuki',
 'swift',
 'strong',
 'hybriddi',
 'gaikindo',
 'indonesia',
 'international',
 'auto',
 'show',
 'giias',
 '2018',
 'merupakan',
 'langkah',
 'lanjutan',
 'dari',
 'pt',
 'suzuki',
 'indomobil',
 'sales',
 'sis',
 'untuk',
 'menghadirkan',
 'kendaraan',
 'ramah',
 'lingkungan',
 'di',
 'tanah',
 'air',
 'pada',
 'gelaran',
 'giias',
 'kali',
 'ini',
 'suzuki',
 'indonesia',
 'menampilkan',
 'teknologi',
 'terkininya',
 'strong',
 'hybrid',
 'pada',
 'diri',
 'suzuki',
 'swift',
 'memang',
 'bukan',
 'yang',
 'pertama',
 'karena',
 'pt',
 'sis',
 'sendiri',
 'sempat',
 'melansir',
 'teknologi',
 'hybrid',
 'ringan',
 'mild',
 'hybrid',
 'pada',
 'suzuki',
 'ertiga',
 'diesel',
 'perbedaannya',
 'strong',
 'hybrid',
 'suzuki',
 'sekarang',
 'sudah',
 'menerapkan',
 'dua',
 'perangkat',
 'tambahan',
 'yaitu',
 'motor',
 'generator',
 'unit',
 'mgu',
 'dan',
 'baterai',
 'lithium',
 'dengan',
 'kemajuan',
 'teknologi',
 'ini',
 'all',
 'new',
 'suzuki

In [109]:
list_corpus = []

In [130]:
# df['content'].apply(lambda x: list_corpus.append(x))

In [111]:
len(list_corpus)

36324

In [112]:
corpus = ' '.join(list_corpus)

In [113]:
tokens = nltk.regexp_tokenize(corpus, pattern)

In [114]:
tokens = ' '.join([' '.join(nltk.regexp_tokenize(word, pattern2)) for word in tokens])

In [115]:
tokens = nltk.word_tokenize(tokens.lower())

In [116]:
len(tokens)

10598226

In [117]:
tokens[:4]

['tangerang', 'honda', 'brio', 'diberikan']

In [118]:
fdist = nltk.FreqDist(ch for ch in tokens if ch.isalpha())

In [119]:
fdist.most_common(5)

[('yang', 248376),
 ('di', 207950),
 ('dan', 196351),
 ('ini', 132105),
 ('dengan', 127581)]

In [121]:
bigrams = ngrams(tokens,2)
trigrams = ngrams(tokens,3)

In [127]:
bigrams = list(ngrams(tokens, 2))

In [128]:
print(*map(' '.join, bigrams), sep=', ')

tangerang honda, honda brio, brio diberikan, diberikan diskon, diskon sebesar, sebesar rp, rp 10, 10 juta, juta diskon, diskon tersebut, tersebut diberikan, diberikan untuk, untuk honda, honda briotipe, briotipe lama, lama selama, selama stok, stok tersebut, tersebut masih, masih ada, ada hal, hal ini, ini disampaikan, disampaikan oleh, oleh salah, salah satu, satu tenaga, tenaga penjual, penjual yang, yang kami, kami temui, temui di, di ajang, ajang gaikindo, gaikindo indonesia, indonesia international, international auto, auto show, show giias, giias 2018, 2018 diskon, diskon honda, honda brio, brio termasuk, termasuk brio, brio satya, satya adalah, adalah sebesar, sebesar rp, rp 10, 10 juta, juta itu, itu berlaku, berlaku untuk, untuk pembelian, pembelian secara, secara kredit, kredit maupun, maupun cash, cash ungkapnya, ungkapnya saat, saat kami, kami temui, temui jumlah, jumlah unit, unit yang, yang tersedia, tersedia pun, pun diklaim, diklaim masih, masih cukup, cukup banyak, ban

, masalah pada, pada penggunaan, penggunaan bahan, bahan bakar, bakar b20, b20 ujar, ujar bob, bob azam, azam direktur, direktur administrasi, administrasi korporasi, korporasi dan, dan hubungan, hubungan eksternal, eksternal pt, pt tmmin, tmmin di, di sela, sela sela, sela pameran, pameran gaikindo, gaikindo indonesia, indonesia international, international auto, auto show, show giias, giias 2018, 2018 dengan, dengan hasil, hasil ini, ini tentunya, tentunya dapat, dapat menyakinkan, menyakinkan pelanggan, pelanggan bahwa, bahwa jika, jika peraturan, peraturan ini, ini berlaku, berlaku mulai, mulai september, september 2018, 2018 nanti, nanti para, para pemilik, pemilik mobil, mobil tidak, tidak akan, akan mengalami, mengalami hambatan, hambatan dan, dan intinya, intinya seluruh, seluruh mobil, mobil dengan, dengan engine, engine diesel, diesel yang, yang diproduksi, diproduksi di, di tmmin, tmmin plant, plant kinerjanya, kinerjanya tetap, tetap handal, handal aman, aman saat, saat men

pt saic, saic general, general motors, motors wuling, wuling wuling, wuling indonesia, indonesia mengatakan, mengatakan bahwa, bahwa wuling, wuling 1, 1 2, 2 liter, liter akhirnya, akhirnya diputuskan, diputuskan meluncur, meluncur akhir, akhir tahun, tahun ini, ini menariknya, menariknya jason, jason menjelaskan, menjelaskan konsep, konsep awal, awal produk, produk telah, telah dirombak, dirombak kami, kami mengubah, mengubah produknya, produknya spesifikasinya, spesifikasinya strukturnya, strukturnya namanya, namanya mobil, mobil ini, ini tetap, tetap 1, 1 2, 2 liter, liter tapi, tapi kami, kami menaruhnya, menaruhnya di, di segmen, segmen berbeda, berbeda ujar, ujar dia, dia menanggapi, menanggapi pertanyaan, pertanyaan mobil123, mobil123 com, com di, di sela, sela sela, sela gaikindo, gaikindo indonesia, indonesia international, international auto, auto show, show giias, giias 2018, 2018 2, 2 12, 12 agustus, agustus di, di indonesia, indonesia convention, convention exhibition, exh

, di iims, iims 2018, 2018 honda, honda secara, secara khusus, khusus mengembangkan, mengembangkan mobil, mobil ini, ini untuk, untuk pasar, pasar indonesia, indonesia all, all new, new briokini, briokini memiliki, memiliki dimensi, dimensi yang, yang lebih, lebih panjang, panjang dan, dan lebar, lebar sehingga, sehingga mampu, mampu kapasitasnya, kapasitasnya lebih, lebih besar, besar baik, baik itu, itu pada, pada bagian, bagian kabin, kabin dan, dan bagasi, bagasi sedangkan, sedangkan untuk, untuk desainnya, desainnya sendiri, sendiri honda, honda tampak, tampak memadukan, memadukan tampilan, tampilan desain, desain khas, khas honda, honda mobilio, mobilio dengan, dengan small, small rs, rs concept, concept perpaduan, perpaduan kedua, kedua gaya, gaya desain, desain tersebut, tersebut mampu, mampu menghasilkan, menghasilkan tampilan, tampilan yang, yang menarik, menarik dinamis, dinamis elegan, elegan dan, dan modern, modern meski, meski beragam, beragam ubahan, ubahan telah, telah 

taman lalu, lalu lintas, lintas bandung, bandung memiliki, memiliki nilai, nilai historis, historis bagi, bagi masyarakat, masyarakat kota, kota bandung, bandung dan, dan hingga, hingga saat, saat ini, ini merupakan, merupakan salah, salah satu, satu tempat, tempat favorit, favorit anak, anak anak, anak dan, dan menjadi, menjadi salah, salah satu, satu tujuan, tujuan wisata, wisata kota, kota bandung, bandung kami, kami berharap, berharap program, program ini, ini dapat, dapat berkelanjutan, berkelanjutan sehingga, sehingga memberikan, memberikan manfaat, manfaat yang, yang besar, besar bagi, bagi masyarakat, masyarakat di, di kota, kota bandung, bandung dan, dan sekitarnya, sekitarnya kata, kata prof, prof yul, yul yunazwin, yunazwin nazaruddin, nazaruddin sebagai, sebagai ketua, ketua pembina, pembina taman, taman lalu, lalu lintas, lintas bandung, bandung sejak, sejak diresmikan, diresmikan tahun, tahun lalu, lalu jumlah, jumlah pengunjungnya, pengunjungnya meningkat, meningkat pesa

...

, 8 ini, ini memungkinkan, memungkinkan konsumsi, konsumsi bahan, bahan bakar, bakar lebih, lebih hemat, hemat sebesar, sebesar 10, 10 hingga, hingga 20, 20 dibandingkan, dibandingkan dengan, dengan teknologi, teknologi konvensional, konvensional tanpa, tanpa mengorbankan, mengorbankan performa, performa mesin, mesin masih, masih ada, ada pula, pula teknologi, teknologi teknologi, teknologi kekinian, kekinian terkait, terkait keselamatan, keselamatan performa, performa dan, dan kenyamanan, kenyamanan xan, xan ari, ari california, california nissan, nissan jukedihentikan, jukedihentikan penjualannya, penjualannya di, di amerika, amerika serikat, serikat keputusan, keputusan untuk, untuk menghentikan, menghentikan penjualan, penjualan nissan, nissan juke, juke karena, karena penjualan, penjualan mobil, mobil tersebut, tersebut semakin, semakin lama, lama semakin, semakin menurun, menurun tercatat, tercatat di, di bulan, bulan juni, juni 2018, 2018 nissan, nissan juke, juke hanya, hanya b

terlalu kencang, kencang saat, saat berjalan, berjalan mundur, mundur g65, g65 yang, yang terkena, terkena recall, recall di, di negeri, negeri paman, paman sam, sam adalah, adalah model, model tertinggi, tertinggi dari, dari generasi, generasi terdahulu, terdahulu jumlah, jumlah unit, unit darikeluarga, darikeluarga g, g classyang, classyang legendaris, legendaris ini, ini di, di as, as seperti, seperti dikabarkan, dikabarkan carscoops, carscoops pada, pada senin, senin 30, 30 7, 7 2018, 2018 hanya, hanya 20, 20 unit, unit biasanya, biasanya kendaraan, kendaraan ditarik, ditarik dan, dan diperbaiki, diperbaiki karena, karena adanya, adanya potensi, potensi kecacatan, kecacatan produksi, produksi seperti, seperti kemungkinan, kemungkinan airbag, airbag tak, tak mengembang, mengembang sempurna, sempurna pemasangan, pemasangan kursi, kursi tak, tak kokoh, kokoh dan, dan lain, lain lain, lain namun, namun g65, g65 menurut, menurut national, national highway, highway traffic, traffic safet

, model klasik, klasik sampai, sampai flagship, flagship bmw, bmw muncul, muncul di, di film, film tersebut, tersebut bmw, bmw melalui, melalui keterangan, keterangan resmi, resmi yang, yang tertera, tertera di, di situs, situs global, global mereka, mereka beberapa, beberapa waktu, waktu lalu, lalu mengumumkan, mengumumkan kerja, kerja sama, sama dengan, dengan paramount, paramount pictures, pictures untuk, untuk sekuel, sekuel teranyar, teranyar mission, mission impossible, impossible sebelum, sebelum ini, ini bmw, bmw telah, telah pamer, pamer keindahan, keindahan seri, seri 7, 7 di, di film, film red, red sparrowgarapan, sparrowgarapan 20th, 20th century, century fox, fox di, di mission, mission impossible, impossible fallout, fallout ada, ada 3, 3 model, model mobil, mobil serta, serta 1, 1 moge, moge bmw, bmw yang, yang siap, siap unjuk, unjuk performa, performa model, model model, model tersebut, tersebut adalah, adalah m5, m5 seri, seri 7, 7 seri, seri 5, 5 klasik, klasik produ

yang berubah, berubah drastis, drastis menurutnya, menurutnya evolusi, evolusi industri, industri otomotif, otomotif tak, tak akan, akan lagi, lagi ditentukan, ditentukan oleh, oleh integrasi, integrasi vertikal, vertikal tapi, tapi oleh, oleh ekosistem, ekosistem berbasis, berbasis platform, platform karena, karena itu, itu sekarang, sekarang adalah, adalah saat, saat yang, yang tepat, tepat bagi, bagi para, para pabrikan, pabrikan global, global dan, dan perusahaan, perusahaan teknologi, teknologi untuk, untuk bekerja, bekerja sama, sama dan, dan memperdalam, memperdalam kolaborasi, kolaborasi secara, secara optimal, optimal ujar, ujar li, li pemegang, pemegang saham, saham terbesar, terbesar daimler, daimler sendiri, sendiri saat, saat ini, ini adalahorang, adalahorang terkaya, terkaya ke, ke 10, 10 di, di china, china yakni, yakni li, li shufu, shufu li, li pada, pada akhir, akhir februari, februari 2018, 2018 menguasai, menguasai 9, 9 69, 69 persen, persen saham, saham daimler, da

KeyboardInterrupt: 